# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'arvc-azureml-project3-experiment-automl'

experiment=Experiment(ws, experiment_name)

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'arvccompute1'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS12_v2", max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


In [3]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("existing dataset found.")

else:
    print("existing dataset not found. creating new one.")
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

existing dataset found.
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":15, "enable_early_stopping":True, "n_cross_validations":5,"max_concurrent_iterations": 5}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT', blocked_models=['XGBoostClassifier'],**automl_settings)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on arvccompute1 with default configuration
Running on remote compute: arvccompute1
Parent Run ID: AutoML_111e941e-8cd6-43c2-b951-a097693639d4

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_111e941e-8cd6-43c2-b951-a097693639d4',
 'target': 'arvccompute1',
 'status': 'Completed',
 'startTimeUtc': '2021-03-21T15:42:38.804108Z',
 'endTimeUtc': '2021-03-21T16:00:42.561973Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'arvccompute1',
  'AMLSettingsJsonString': '{"path":null,"name":"arvc-azureml-project3-experiment-automl","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-141038","workspace_name":"quick-starts-ws-141038","region":"southcentralus","compute_target":"arvccompute1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validati

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: arvc-azureml-project3-experiment-automl,
Id: AutoML_111e941e-8cd6-43c2-b951-a097693639d4_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
            

In [35]:
best_run_metrics

{'AUC_micro': 0.9263153249668401,
 'precision_score_weighted': 0.8717520327573605,
 'f1_score_micro': 0.8708080808080808,
 'average_precision_score_micro': 0.9286082169460457,
 'accuracy': 0.8708080808080808,
 'AUC_weighted': 0.9128422764511473,
 'f1_score_weighted': 0.8695993209404975,
 'norm_macro_recall': 0.6862606479541963,
 'AUC_macro': 0.9128422764511473,
 'balanced_accuracy': 0.8431303239770982,
 'precision_score_macro': 0.851615640667511,
 'recall_score_weighted': 0.8708080808080808,
 'log_loss': 0.3664475831329427,
 'recall_score_micro': 0.8708080808080808,
 'average_precision_score_weighted': 0.91954924262132,
 'matthews_correlation': 0.6941205997015152,
 'f1_score_macro': 0.8450455818691113,
 'precision_score_micro': 0.8708080808080808,
 'average_precision_score_macro': 0.898244043619871,
 'weighted_accuracy': 0.8889202246662539,
 'recall_score_macro': 0.8431303239770982,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_111e941e-8cd6-43c2-b951-a097693639d4_38

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [47]:
#TODO: Save the best model
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='arvc_automl_model3',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})


# List registered models
for m in Model.list(ws):
    print(m.name, 'version:', m.version)
    for tag_name in m.tags:
        tag = m.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in m.properties:
        prop = m.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

arvc_automl_model3 version: 1
	 Training context : Auto ML
	 AUC : 0.9128422764511473
	 Accuracy : 0.8708080808080808


arvc_automl_model2 version: 1
	 Training context : Auto ML
	 AUC : 0.9128422764511473
	 Accuracy : 0.8708080808080808


arvc_automl_model version: 4
	 Training context : Auto ML
	 AUC : 0.9128422764511473
	 Accuracy : 0.8708080808080808


arvc_automl_model version: 3
	 Training context : Auto ML
	 AUC : 0.9128422764511473
	 Accuracy : 0.8708080808080808


arvc_best_hyperdrive_model5 version: 1




In [48]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [50]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="scoring.py")

In [51]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [62]:
mybestmodel = ws.models["arvc_automl_model3"]

In [64]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="arvc-automl-best-model-svc6",
                    models=[mybestmodel],
                    inference_config=inference_config
                    #deployment_config=deployment_config
                    )
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-21 16:50:35+00:00 Creating Container Registry if not exists.
2021-03-21 16:50:38+00:00 Use the existing image.
2021-03-21 16:50:39+00:00 Generating deployment configuration.
2021-03-21 16:50:41+00:00 Submitting deployment to compute..
2021-03-21 16:50:46+00:00 Checking the status of deployment arvc-automl-best-model-svc6.

TODO: In the cell below, print the logs of the web service and delete the service

In [45]:
print(service.get_logs())

None


In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
import requests
import json

data={"data":
  [{'age': 87.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 0,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 0,
  'smoking': 0,
  'time': 4}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

In [ ]:
result.status_code

In [ ]:
service.delete()
compute_target.delete()
model.delete()
run.delete()
experiment.delete()